In [ ]:
ml_catalog = dbutils.widgets.get("ml_catalog")
ml_search_db = dbutils.widgets.get("ml_search_db")

df_people = spark.sql(f"""
    SELECT details_search_value, COUNT(*) AS frequency
    FROM {ml_catalog}.{ml_search_db}.ml_search
    WHERE action = 'people'
    GROUP BY details_search_value
    ORDER BY frequency DESC
    LIMIT 10
""")

df_action = spark.sql(f"""
    SELECT details_search_value, COUNT(*) AS frequency
    FROM {ml_catalog}.{ml_search_db}.ml_search
    WHERE action = 'actions'
    GROUP BY details_search_value
    ORDER BY frequency DESC
    LIMIT 10
""")

df = df_people.union(df_action)

### NLP Spacy Intent Detector

In [ ]:
import spacy

In [ ]:
%pip install ../intent_detector_model/en_core_web_sm-3.7.1.tar.gz

In [ ]:
%pip install ../intent_detector_model/xx_ent_wiki_sm-3.7.0.tar.gz

In [ ]:
from pathlib import Path
import sys

current_dir = Path.cwd()
parent_dir = current_dir.parent.parent
sys.path.append(str(parent_dir))

In [ ]:
from search.intent_detector_model.intentDetector import IntentDetector
import pandas as pd

conf = {}
conf["action_data"] = pd.read_csv("../data/action_data.txt")["word"].to_list()
conf["people_data"] = pd.read_csv("../data/people_data.txt")["word"].to_list()
conf["position_data"] = pd.read_csv("../data/position_data.txt")["word"].to_list()
conf["common_action_typos"] = pd.read_csv("../data/common_action_typos.txt")["word"].to_list()
detector = IntentDetector(conf)
detector.predictModel("Manage Associate's pay data")

In [ ]:
pdf = df.toPandas()
pdf["intent"] = pdf["details_search_value"].apply(lambda x: detector.predictModel(x))
display(pdf)